In [34]:
import json
import requests

import asyncio
import aiohttp

import pandas as pd
import numpy as np


CLIENT_ID = ""
CLIENT_SECRET = ""
ACCESS_TOKEN = ""
REFRESH_TOKEN = ""
BASE_URL = 'http://www.zumsoon.com/oauth/v1.0/'
ACCESS_TOKEN_URL='http://www.zumsoon.com/oauth/token'


def set_credentials(client_id, client_secret, access_token, refresh_token):
    global ACCESS_TOKEN
    global REFRESH_TOKEN
    global CLIENT_ID
    global CLIENT_SECRET

    ACCESS_TOKEN = access_token
    REFRESH_TOKEN = refresh_token
    CLIENT_ID = client_id
    CLIENT_SECRET = client_secret


def refresh_token():
    
    global ACCESS_TOKEN
    global REFRESH_TOKEN
    global CLIENT_ID
    global CLIENT_SECRET
                  
    data = dict()
    data['grant_type'] = 'refresh_token'
    data['refresh_token'] = REFRESH_TOKEN
    data['client_id'] = CLIENT_ID
    data['client_secret'] = CLIENT_SECRET
    # make custom POST request to get the new token pair
    resp = requests.post(ACCESS_TOKEN_URL, data=data)
    resp = resp.json()
    
    ACCESS_TOKEN = resp['access_token']
    REFRESH_TOKEN = resp['refresh_token']
    
    # checks the response status and parses the new tokens
    # if refresh failed will redirect to login

    return resp

def get_token():
    resp = {'access_token': ACCESS_TOKEN,
           'refresh_token': REFRESH_TOKEN}
    print(resp)

def request(api, method='GET', json=None):
    token = ACCESS_TOKEN
    headers = {"Authorization": "Bearer " + token}
    if method == 'GET':
        resp = requests.get(BASE_URL+api, headers=headers)
        return resp
    
    elif method == 'POST':
        resp = requests.post(BASE_URL+api, headers=headers, json=json)
        return resp
#         if data is not None:
#             resp = requests.post(BASE_URL+api, headers=headers, data=data)
#             return resp
#         elif json is not None:
#             resp = requests.post(BASE_URL+api, headers=headers, json=json)
#             return resp
#         else:
#             return 'data is none.'
        
    else:
        pass

def user_me():
    api = "user/me"
    resp = request(api)
    return resp.json()


def user_client():
    api = "user/client"
    resp = request(api)
    return resp.json()


def user_new_order_messages():
    api = "user/new-order-messages"
    resp = request(api)
    data = resp.json()
    data = json.dumps(data)
    df = pd.read_json(data, orient='records')
    return df

def account():
    api = "account"
    resp = request(api)
    data = resp.json()
    data = json.dumps(data)
    df = pd.read_json(data, orient='records')
    return df


def account_ledger(method='GET', txs=None):
    api = "account/ledger"
    if method == 'GET':
        resp = request(api)
        data = resp.json()
        data = json.dumps(data)
        df = pd.read_json(data, orient='records')
        return df
    elif method == 'POST':
#         txs = json.dumps(txs)
#         resp = request(api, method='POST', data={'txs': txs})
#         return resp.json()
        resp = request(api, method='POST', json=txs)
        return resp.json()
    

def account_report(report_name):
    api = "account/report/{}".format(report_name)
    resp = request(api)
    data = resp.json()
    data = json.dumps(data)
    df = pd.read_json(data, orient='records')
    return df

def account_financial_assets():
    api = "account/financial-assets"
    resp = request(api)
    data = resp.json()
    data = json.dumps(data)
    df = pd.read_json(data, orient='records')
    return df

def broker():
    api = "broker"
    resp = request(api)
    return resp.json()


def broker_current_quote(commodity):
    api = 'broker/current-quote/{}'.format(commodity)
    resp = request(api)
    return resp.json()


def broker_lob(commodity):
    api = 'broker/lob/{}'.format(commodity)
    resp = request(api)
    if resp.status_code == 200:
        lob = resp.json()
        _lob_ask = {int(k): v for k, v in lob['ASK'].items()}
        _lob_bid = {int(k): v for k, v in lob['BID'].items()}

        lob_ask = []
        for key in sorted(_lob_ask.keys()):
            lob_ask.append([key, _lob_ask[key][0], _lob_ask[key][1]])

        lob_bid = []
        for key in sorted(_lob_bid.keys(), reverse=True):
            lob_bid.append([key, _lob_bid[key][0], _lob_bid[key][1]])

        return [lob_ask, lob_bid]
    else:
        return resp.json()

def broker_order(method='GET', order=None):
    """
    order = {'commodity': string, 'quote': integer, 'qty': integer}
            (qty<0 -> ask, qty>0 -> bid)
    Only normal order is allowed, not market order!
    """
    api = 'broker/order'
    if method == 'GET':
        resp = request(api)
        data = resp.json()
        data = json.dumps(data)
        df = pd.read_json(data, orient='records')
        return df
    elif method == 'POST':
        resp = request(api=api, method='POST', json=order)
        return resp.json()

def broker_cancel(cancel):
    api = 'broker/cancel'
    resp = request(api=api, method='POST', json=cancel)
    return resp.json()


def broker_chart(commodity):
    api = "broker/chart/{}".format(commodity)
    resp = request(api)
    if resp.status_code == 200:
        data = resp.json()
        df = pd.DataFrame.from_dict(data)
        return df
    else:
        return resp.json()

In [35]:
set_credentials(client_id="SPfI9VI4odL4wtAse1DDqylV1nI2RzNW5MPon8SG", 
                client_secret="bM2YkUV4zp26dZIJ73EbeHgKEEUcQTFn3yrBLvpLySdLeoLbd9", 
                access_token="3AL4Ddzscl2UsKjuZByviD2tTHaHOV", 
                refresh_token="YjiyBIvOgIYLozTVRHj501eICneYhC")

In [36]:
refresh_token()

{'access_token': 'pCWlVqPK6UY4QB7tKIb9TEKq4xPMXx',
 'refresh_token': '9tijZRD3igEDzP2dqK5XFkKFUueDBx',
 'scope': 'email address',
 'token_type': 'Bearer'}

In [37]:
user_me()

{'email': 'happydhseo@kaist.ac.kr',
 'first_name': 'Donghyun',
 'last_name': 'Seo',
 'username': 'dhseo'}

In [4]:
user_client()

{'client_id': 'SPfI9VI4odL4wtAse1DDqylV1nI2RzNW5MPon8SG',
 'client_name': None,
 'client_scopes': ['email', 'address']}

In [6]:
user_new_order_messages()

,bid_ask,commodity,dt,event,order_id,qty,quote
0,ASK,12.0,"Mon, 26 Feb 2018 23:41:51 GMT",accept,2018-02-26:7,5.0,96.0
1,ASK,12.0,"Mon, 26 Feb 2018 23:41:51 GMT",conclude,2018-02-26:7,5.0,98.0
2,NaN,NaN,"Mon, 26 Feb 2018 23:41:51 GMT",complete,2018-02-26:7,NaN,NaN
3,BID,12.0,"Mon, 26 Feb 2018 23:41:51 GMT",accept,2018-02-26:14,6.0,103.0
4,BID,12.0,"Mon, 26 Feb 2018 23:41:51 GMT",conclude,2018-02-26:14,5.0,102.0
5,ASK,12.0,"Mon, 26 Feb 2018 23:41:51 GMT",accept,2018-02-26:15,9.0,95.0
6,BID,12.0,"Mon, 26 Feb 2018 23:41:51 GMT",conclude,2018-02-26:14,1.0,103.0
7,NaN,NaN,"Mon, 26 Feb 2018 23:41:51 GMT",complete,2018-02-26:14,NaN,NaN
8,ASK,12.0,"Mon, 26 Feb 2018 23:41:51 GMT",conclude,2018-02-26:15,1.0,103.0
9,ASK,12.0,"Mon, 26 Feb 2018 23:41:51 GMT",conclude,2018-02-26:15,4.0,100.0


In [40]:
account()

,code,description,name,parent_account
0,1000,자산,asset,ASSET
1,1001,현금,cash,ASSET
2,1002,금융자산,Financial Assets,ASSET
3,2000,부채,liability,LIABILITY
4,3000,자본,capital,CAPITAL
5,3001,자본잉여금,Additional Paid-in and Other Capital,CAPITAL
6,3002,자본조정,Capital Adjustments,CAPITAL
7,3003,기타포괄손익누계액,Accumulated Other Comprehensive Income,CAPITAL
8,3004,이익잉여금,Retained Earnings,CAPITAL
9,4000,수익,income,INCOME


In [9]:
account_ledger()

,account,account_description,account_name,account_parent_account,commodity,commodity_name,commodity_unit,dt,id,price,price_base_commodity,qty,tx_desc,tx_desc_code,tx_id
0,1001,현금,cash,ASSET,1,KRW,WON,"Thu, 08 Mar 2018 02:06:05 GMT",3133,1,1,10000.00,trade test 0,10,453
1,3000,자본,capital,CAPITAL,1,KRW,WON,"Thu, 08 Mar 2018 02:06:05 GMT",3134,1,1,10000.00,trade test 0,10,453
2,1002,금융자산,Financial Assets,ASSET,12,game_stock_2,EA,"Thu, 08 Mar 2018 02:06:05 GMT",3135,99,1,6.00,trade test 1,10,453
3,1001,현금,cash,ASSET,1,KRW,WON,"Thu, 08 Mar 2018 02:06:05 GMT",3136,1,1,-594.59,trade test 1,10,453
4,5002,수수료,fees,EXPENSE,1,KRW,WON,"Thu, 08 Mar 2018 02:06:05 GMT",3137,1,1,0.59,trade test 1,10,453
5,1002,금융자산,Financial Assets,ASSET,12,game_stock_2,EA,"Thu, 08 Mar 2018 02:06:05 GMT",3138,99,1,-6.00,trade test 2,10,453
6,1001,현금,cash,ASSET,1,KRW,WON,"Thu, 08 Mar 2018 02:06:05 GMT",3139,1,1,593.41,trade test 2,10,453
7,5002,수수료,fees,EXPENSE,1,KRW,WON,"Thu, 08 Mar 2018 02:06:05 GMT",3140,1,1,0.59,trade test 2,10,453
8,1001,현금,cash,ASSET,1,KRW,WON,"Thu, 08 Mar 2018 02:06:05 GMT",3141,1,1,10000.00,trade test 0,10,454
9,3000,자본,capital,CAPITAL,1,KRW,WON,"Thu, 08 Mar 2018 02:06:05 GMT",3142,1,1,10000.00,trade test 0,10,454


In [10]:
txs = [
        {'tx_desc': 'trade test 0',
         'split':
             [
                 {'account': 1001, 'commodity': 1, 'qty': 10000, 'price': 1, 'price_base_commodity': 1},
                 {'account': 3000, 'commodity': 1, 'qty': 10000, 'price': 1, 'price_base_commodity': 1}
             ]
         },
        {'tx_desc': 'trade test 1',
         'split':
             [
                 {'account': 1002, 'commodity': 12, 'qty': 6, 'price': 99, 'price_base_commodity': 1},
                 {'account': 1001, 'commodity': 1, 'qty': -594.59, 'price': 1, 'price_base_commodity': 1},
                 {'account': 5002, 'commodity': 1, 'qty': 0.59, 'price': 1, 'price_base_commodity': 1}
             ]
         },
        {'tx_desc': 'trade test 2',
         'split':
             [
                 {'account': 1002, 'commodity': 12, 'qty': -6, 'price': 99, 'price_base_commodity': 1},
                 {'account': 1001, 'commodity': 1, 'qty': 593.41, 'price': 1, 'price_base_commodity': 1},
                 {'account': 5002, 'commodity': 1, 'qty': 0.59, 'price': 1, 'price_base_commodity': 1}
             ]
         }
    ]

order = {'commodity': 11, 'quote': 1700000, 'qty': -3}
cancel = {'commodity': 11, 'order_id': '2018-03-06:14'}

In [11]:
account_ledger(method='POST', txs=txs)

{'status': 'accepted'}

In [12]:
account_report('balance-sheet')

,account,commodity,qty,value
0,현금,KRW,-20407523.9,-20407523.9
1,금융자산,game_stock_1,0.0,0.0
2,금융자산,game_stock_2,0.0,0.0
3,자본,KRW,1050000.0,1050000.0
4,수수료,KRW,21457523.9,21457523.9


In [13]:
account_financial_assets()

,commodity,qty,value
0,game_stock_1,0,0
1,game_stock_2,0,0


In [14]:
broker()

[11, 12]

In [43]:
broker_order(method='POST', order=order)

'Order received'

In [44]:
broker_order()

,bid_ask,commodity,conclusion,init_qty,order_id,quote,status
0,ASK,11,0,3,2018-03-12:3,1700000,accepted


In [45]:
cancel = {'commodity': 11, 'order_id': '2018-03-12:3'}
broker_cancel(cancel)

'Canceled'

In [26]:
broker_chart(11)

,close,dt,high,low,open
0,1500000.0,2018-03-06 06:30:00,1500000.0,1500000.0,1500000.0
1,NaN,2018-03-06 06:45:00,NaN,NaN,NaN
2,NaN,2018-03-06 07:00:00,NaN,NaN,NaN
3,NaN,2018-03-06 07:15:00,NaN,NaN,NaN
4,NaN,2018-03-06 07:30:00,NaN,NaN,NaN
5,NaN,2018-03-06 07:45:00,NaN,NaN,NaN
6,NaN,2018-03-06 08:00:00,NaN,NaN,NaN
7,NaN,2018-03-06 08:15:00,NaN,NaN,NaN
8,1700000.0,2018-03-06 08:30:00,1700000.0,1700000.0,1700000.0
9,NaN,2018-03-06 08:45:00,NaN,NaN,NaN
